In [ ]:
import pandas as pd

In [ ]:
# histórico
df_hist = pd.read_parquet("loans_history.parquet")

gen = (
    LookAhead(
        id_cols=["contract_id"],
        date_cols=["booking_date"],
        keep_cols=["pd_observed"],
        ignore_cols=["target_pd"],
        random_state=42,
    )
    .fit(df_hist)
)

# choques de stress
shocks_cfg = {
    "days_past_due": {"Δμ": 7, "stretch": 1.4},
    "utilization_ratio": {
        "quantile_fn": lambda q: np.minimum(1, q ** 0.7)
    },
}

df_future = gen.generate(
    n_periods=6,
    freq="M",
    start_vintage="2025-07-01",
    scenario="stress",
    shocks=shocks_cfg,
    n_per_vintage=5000,
)

gen.validate(df_future)
print(df_future.head())
